# Dataset Handling

#### Import Libraries

In [84]:
import pandas as pd
import numpy as np

#### Import Tables

In [85]:
claims_df = pd.read_csv("Merged Claims.csv") # Claims Data
product_code = pd.read_csv("Product Mapping.csv") # Product Mapping Code
procedure_code = pd.read_csv("Procedure Mapping.csv",dtype='object') # Procedure Mapping Code
product_category =  pd.read_csv("Product Category.csv") # Product Category
product_abbreviation =  pd.read_csv("Product Abbreviation.csv") # Product Abbreviation
mNSCLC_List = pd.read_csv('Synthetic mNSCLC Patients.csv') # mNSCLC Patient List
claims_df['Transaction date'] = pd.to_datetime(claims_df['Transaction date'], format='%Y-%m-%d') # Converting date into datetime format

In [86]:
claims_df.head()

,Pat_ID,HCP_ID,Claim ID,HCPCS_ code,Transaction date,Diagnosis 1,Diagnosis 2,Diagnosis 3,Diagnosis 4
0,PATIENT_001,HCP_17,2803,J1004,2022-01-12,C34.11,C78,S45901S,NaN
1,PATIENT_001,HCP_17,6213,J1001,2022-01-12,C34.11,C78,S45901S,NaN
2,PATIENT_001,HCP_17,6867,J1001,2022-02-11,C34.11,C78,N981,NaN
3,PATIENT_001,HCP_17,5629,J1001,2022-03-08,C34.11,C78,S55091A,NaN
4,PATIENT_001,HCP_17,8726,J1001,2022-04-12,C34.11,C78,S45901S,NaN


In [87]:
claims_df['Pat_ID'].unique

<bound method Series.unique of 0       PATIENT_001
1       PATIENT_001
2       PATIENT_001
3       PATIENT_001
4       PATIENT_001
           ...     
5296    PATIENT_425
5297    PATIENT_425
5298    PATIENT_425
5299    PATIENT_425
5300    PATIENT_425
Name: Pat_ID, Length: 5301, dtype: object>

#### Visualize Table

In [88]:
claims_df.head()

,Pat_ID,HCP_ID,Claim ID,HCPCS_ code,Transaction date,Diagnosis 1,Diagnosis 2,Diagnosis 3,Diagnosis 4
0,PATIENT_001,HCP_17,2803,J1004,2022-01-12,C34.11,C78,S45901S,NaN
1,PATIENT_001,HCP_17,6213,J1001,2022-01-12,C34.11,C78,S45901S,NaN
2,PATIENT_001,HCP_17,6867,J1001,2022-02-11,C34.11,C78,N981,NaN
3,PATIENT_001,HCP_17,5629,J1001,2022-03-08,C34.11,C78,S55091A,NaN
4,PATIENT_001,HCP_17,8726,J1001,2022-04-12,C34.11,C78,S45901S,NaN


In [89]:
product_code.head()

,HCPCS_ code,Product Name
0,J1001,Ticaprant
1,J1002,NOVmatinib
2,J1003,AZNertrib
3,J1004,BMSRizi
4,J2001,Pemetrexed


In [90]:
procedure_code.head()

,HCPCS_ code,Procedure Name
0,81235,EGFR test
1,78815,PET scan
2,70450,Brain MRI
3,70460,Brain MRI
4,70470,Brain MRI


In [91]:
product_abbreviation.head()

,Product Name,Product Abbreviation
0,Ticaprant,TIC
1,NOVmatinib,NOV
2,AZNertrib,ERT
3,BMSRizi,RIZ


In [92]:
mNSCLC_List.head()

,mNSCLC_Patients
0,PATIENT_001
1,PATIENT_002
2,PATIENT_003
3,PATIENT_004
4,PATIENT_005


#### Joining Tables

In [93]:
mapped_df = pd.merge(pd.merge(pd.merge(pd.merge(claims_df,product_code, how = 'left', on = 'HCPCS_ code'),procedure_code, how = 'left', on = 'HCPCS_ code'),product_category, how = 'left', on = 'Product Name'), product_abbreviation, how = 'left', on = 'Product Name')
mapped_df.head()

,Pat_ID,HCP_ID,Claim ID,HCPCS_ code,Transaction date,Diagnosis 1,Diagnosis 2,Diagnosis 3,Diagnosis 4,Product Name,Procedure Name,Product Category,Product Abbreviation
0,PATIENT_001,HCP_17,2803,J1004,2022-01-12,C34.11,C78,S45901S,NaN,BMSRizi,NaN,Market Basket,RIZ
1,PATIENT_001,HCP_17,6213,J1001,2022-01-12,C34.11,C78,S45901S,NaN,Ticaprant,NaN,Market Basket,TIC
2,PATIENT_001,HCP_17,6867,J1001,2022-02-11,C34.11,C78,N981,NaN,Ticaprant,NaN,Market Basket,TIC
3,PATIENT_001,HCP_17,5629,J1001,2022-03-08,C34.11,C78,S55091A,NaN,Ticaprant,NaN,Market Basket,TIC
4,PATIENT_001,HCP_17,8726,J1001,2022-04-12,C34.11,C78,S45901S,NaN,Ticaprant,NaN,Market Basket,TIC


#### Dataset Prepration (Reference to Patient Cohort Analysis)

In [94]:
# NSCLC Patients
mNSCLC_list = mNSCLC_List['mNSCLC_Patients'].tolist()
mNSCLC_patients = mapped_df[mapped_df['Pat_ID'].isin(mNSCLC_list)]

# Market Basket mNSCLC Patients
#market_mNSCLC_pat = mNSCLC_patients[mNSCLC_patients['Product Category'] == 'Market Basket']

In [95]:
raw_df = mNSCLC_patients

#### Outlier Detection and Removal

In [96]:
raw_df.shape

(5040, 13)

In [97]:
# Outlier Detection
flag = raw_df.groupby(['Pat_ID','HCPCS_ code'])['HCPCS_ code'].agg(
    flag = lambda x: x.count()
)

flagged_df = raw_df.merge(flag, how = 'left', on = ['Pat_ID','HCPCS_ code'])

In [98]:
flagged_df

,Pat_ID,HCP_ID,Claim ID,HCPCS_ code,Transaction date,Diagnosis 1,Diagnosis 2,Diagnosis 3,Diagnosis 4,Product Name,Procedure Name,Product Category,Product Abbreviation,flag
0,PATIENT_001,HCP_17,2803,J1004,2022-01-12,C34.11,C78,S45901S,NaN,BMSRizi,NaN,Market Basket,RIZ,3
1,PATIENT_001,HCP_17,6213,J1001,2022-01-12,C34.11,C78,S45901S,NaN,Ticaprant,NaN,Market Basket,TIC,7
2,PATIENT_001,HCP_17,6867,J1001,2022-02-11,C34.11,C78,N981,NaN,Ticaprant,NaN,Market Basket,TIC,7
3,PATIENT_001,HCP_17,5629,J1001,2022-03-08,C34.11,C78,S55091A,NaN,Ticaprant,NaN,Market Basket,TIC,7
4,PATIENT_001,HCP_17,8726,J1001,2022-04-12,C34.11,C78,S45901S,NaN,Ticaprant,NaN,Market Basket,TIC,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,PATIENT_425,HCP_77,2467,70552,2022-08-18,C34.2,C78.4,NaN,NaN,NaN,Brain MRI,NaN,NaN,2
5036,PATIENT_425,HCP_77,2468,78815,2022-09-18,I10,C78.7,NaN,NaN,NaN,PET scan,NaN,NaN,1
5037,PATIENT_425,HCP_77,2469,81235,2022-10-04,C34.1,C78.3,NaN,NaN,NaN,EGFR test,NaN,NaN,1
5038,PATIENT_425,HCP_77,2470,70551,2022-06-05,C34.1,C78.8,NaN,NaN,NaN,Brain MRI,NaN,NaN,1


In [99]:
# Outlier Removal
df = flagged_df[flagged_df['flag'] != 1]

In [100]:
df.shape

(3223, 14)

In [101]:
raw_df['Pat_ID'].unique

<bound method Series.unique of 0       PATIENT_001
1       PATIENT_001
2       PATIENT_001
3       PATIENT_001
4       PATIENT_001
           ...     
5296    PATIENT_425
5297    PATIENT_425
5298    PATIENT_425
5299    PATIENT_425
5300    PATIENT_425
Name: Pat_ID, Length: 5040, dtype: object>

# Claims Data Processing

#### Filter out year-month as a new column

In [102]:
# Filter out year-month
df['YM'] = df['Transaction date'].apply(lambda x:x.strftime('%Y%m'))

/var/folders/sg/3tj_k0kn1315z20ckp9dznph0000gn/T/ipykernel_55794/2011112001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['YM'] = df['Transaction date'].apply(lambda x:x.strftime('%Y%m'))


#### Calculation of Gap

In [70]:
'''# Calculation of Gap
grouped = df.groupby('Transaction date')
df_sorted = grouped.apply(lambda x:x.sort_values(by='Transaction date')).reset_index(drop = True)
df_sorted['Gap'] = df_sorted.groupby('Pat_ID')['Transaction date'].diff().fillna(pd.Timedelta(0)).dt.days
df_with_gap = df_sorted'''

"# Calculation of Gap\ngrouped = df.groupby('Transaction date')\ndf_sorted = grouped.apply(lambda x:x.sort_values(by='Transaction date')).reset_index(drop = True)\ndf_sorted['Gap'] = df_sorted.groupby('Pat_ID')['Transaction date'].diff().fillna(pd.Timedelta(0)).dt.days\ndf_with_gap = df_sorted"

In [103]:
df_sorted = df.sort_values(by=['Pat_ID', 'Transaction date']).reset_index(drop=True)
df_sorted['Gap'] = df_sorted.groupby('Pat_ID')['Transaction date'].diff().fillna(pd.Timedelta(0)).dt.days
df_with_gap = df_sorted

#### LOT Creation

In [112]:
'''# Initialize the Lot Number
df_with_gap['Lot number'] = 1

# Function to update the 'Lot Number' based on the gap condition
def update_lot_number(sub_df):
    current_value = 1
    for i in range(1, len(sub_df)):
        if sub_df.iloc[i]['Gap'] > 60:
            current_value += 1
        sub_df.at[sub_df.index[i], 'Lot number'] = current_value
    return sub_df

# Apply the function to each group
df_with_LOT = df_with_gap.groupby('Pat_ID').apply(update_lot_number).reset_index(drop=True)'''

/var/folders/sg/3tj_k0kn1315z20ckp9dznph0000gn/T/ipykernel_55794/1075989800.py:14: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_with_LOT = df_with_gap.groupby('Pat_ID').apply(update_lot_number).reset_index(drop=True)


In [ ]:
# Initialize the Lot Number
df_with_gap['Lot number'] = 1

# Function to update the 'Lot Number' based on the gap condition
def update_lot_number_limited(sub_df):
    current_value = 1
    for i in range(1, len(sub_df)):
        if sub_df.iloc[i]['Gap'] > 60 and current_value < 2:  # Limit to 2 LoTs
            current_value += 1
        sub_df.at[sub_df.index[i], 'Lot number'] = current_value
    return sub_df

# Apply the function to each group
df_with_LOT = df_with_gap.groupby('Pat_ID').apply(update_lot_number_limited).reset_index(drop=True)


In [113]:
df_with_LOT['Lot number'].value_counts()

2    1928
1     701
3     550
4      42
5       2
Name: Lot number, dtype: int64

#### Adding Date Feature
- Start Date
- End Date

In [73]:
# Calculate start date and end date
date_df = df_with_LOT.groupby(['Pat_ID','Lot number']).agg(
                                        start_date = ('Transaction date','min'),
                                        end_date = ('Transaction date','max')
)

# Restructure Date Format to mmddyyyy
def convert_to_mmddyyyy(date):
    return date.strftime('%m-%d-%Y')

# Apply the function to the 'date' column
date_df['start_date'] = date_df['start_date'].apply(convert_to_mmddyyyy)
date_df['end_date'] = date_df['end_date'].apply(convert_to_mmddyyyy)
df_with_LOT['Transaction date'] = df_with_LOT['Transaction date'].apply(convert_to_mmddyyyy)

#### Joining the date features to the main table

In [74]:
# Join the date columns to the main table
merged_df_date = df_with_LOT.merge(date_df,how = 'left', on=['Pat_ID','Lot number'])

#### Regimen Calculation

In [75]:
## Regimen Calculation
df_regimen = merged_df_date.groupby(['Pat_ID','Lot number'])['Product Abbreviation'].apply(lambda x: '_'.join(sorted(set(x.dropna()), key = lambda prod:prod[0]))).reset_index()
df_regimen.rename(columns={'Product Abbreviation':'Regimen'}, inplace=True)

#### Joining Regimen table to the main table

In [76]:
# Join Regimen table to main table
claim_level_data = merged_df_date.merge(df_regimen,how = 'left', on=['Pat_ID','Lot number'])

#### Saving File into CSV

In [77]:
# Saving File
claim_level_data.to_csv("Synthetic claim_level_data.csv", index = None)

# Patients x LOT level data Creation

In [78]:
# LOT creation
claim_count = claim_level_data.groupby(['Pat_ID','Lot number','HCP_ID','Regimen'])['HCPCS_ code'].agg(
    claim_count = lambda x : x.count()
)

In [79]:
LOT_working_table = claim_level_data.groupby(['Pat_ID','Lot number','HCP_ID','Regimen']).agg(
    start_date = ('start_date','min'),
    end_date = ('end_date','max')
)

In [80]:
# Final LOT level data
Patient_LOT_data = LOT_working_table.merge(claim_count, on = ['Pat_ID','Lot number','HCP_ID','Regimen'], how = 'left')

In [81]:
# Saving file in csv
Patient_LOT_data.to_csv("Synthetic Patient_LOT_data.csv")